In [1]:
import pandas as pd
import numpy as np
import pydit


In [2]:
d1 = {
        "mkey": [1, 2, 3,4],
        "mvalue": ["a","b","c","d"]
    }
d2 = {
        "tkey": [100, 101, 102,103,104,105,106],
        "mkey": [1,1,1,2,3,3,5],
        "tvalue": [10.12,20.4,33.3,45,59.99,60,-1]
    }

df1 = pd.DataFrame(data=d1)
df2= pd.DataFrame(data=d2)

In [3]:
def add_countif(df1,df2,left_on=None,right_on=None,on=None):
    if on:
        left_on=on
        right_on=on
    if (not left_on) or (not right_on):
        print("Missing key")
        return None
    #df1["count"]= df1['mkey'].map(df2.groupby('mkey')['mkey'].count())
    #df2["count"]= df2['mkey'].map(df1.groupby('mkey')['mkey'].count())
    df1["count_"+right_on]= df1['mkey'].map(df2[right_on].value_counts())
    df2["count_"+left_on]= df2['mkey'].map(df1[left_on].value_counts())
    df1["count_"+right_on]=df1["count_"+left_on].fillna(0).astype('Int64')
    df2["count_"+left_on]=df2["count_"+right_on].fillna(0).astype('Int64')
add_countif(df1, df2, on="mkey")

In [4]:
df1

,mkey,mvalue,count_mkey
0,1,a,3
1,2,b,1
2,3,c,2
3,4,d,0


In [5]:
df2

,tkey,mkey,tvalue,count_mkey
0,100,1,10.12,1
1,101,1,20.40,1
2,102,1,33.30,1
3,103,2,45.00,1
4,104,3,59.99,1
5,105,3,60.00,1
6,106,5,-1.00,0


In [16]:
import pandas as pd
import numpy as np
import random
d1 = {
        "mkey": ["a", "b", "c","d",np.nan],
        "mvalue": ["aa","bb","cc","dd",""]
    }
d2 = {
        "tkey": [100, 101, 102,103,104,105,106,107],
        "mkey": ["a","a","a","b","c","c","e",np.nan],
        "tvalue": [10.12,20.4,33.3,45,59.99,60,-1,-1]
    }
d3= {
        "tkey": [108, 109, 110,111,112,113,114,115],
        "mkey": ["b","a","b","b","b","e","e","f"],
        "tvalue": [13,19.04,10.3,50,51.99,60,10,4.20]
    }


df1 = pd.DataFrame(data=d1)
df2= pd.DataFrame(data=d2)
df3=pd.DataFrame(data=d3)


def _anonymise(df_list,key_list,map_dict=None,hash_table=None):
    '''Internal function to perform the actual anonymisation'''
    master_list=[]
    for i,df in enumerate(df_list):
        keys=df[key_list[i]]
        master_list.extend(keys)
    unique_keys=set(master_list)
    print(list(unique_keys))
    if map_dict:
        map_dict_new=map_dict.copy()
        cur_keys=set(map_dict.keys())
        new_keys=unique_keys-cur_keys
        dummy_keys=list(range(len(cur_keys)+1,len(cur_keys)+len(new_keys)+1))
    else:
        print("Creating new mapping dictionary")
        map_dict_new={}
        new_keys=list(unique_keys)
        dummy_keys=list(range(1,len(unique_keys)+1))
    print("New keys",new_keys)
    print("Dummy keys",dummy_keys)
    random.shuffle(dummy_keys)
    print(dummy_keys)
    add_list=list(zip(new_keys,dummy_keys))
    add_dict=dict(add_list)
    map_dict_new.update(add_dict)
    for i,df in enumerate(df_list):
        if hash_table:
            df[key_list[i]+"_anon"]=df[key_list[i]].map(lambda x: hash_table[map_dict_new[x]])
                
        else:
            df[key_list[i]+"_anon"]=df[key_list[i]].map(lambda x: map_dict_new[x])
    
    return map_dict_new

def anonymise_dataframe(df_list,key_list, map_table_or_dict=None,hash_list=None,create_new_hash_list=False):
    translation={}
    if create_new_hash_list:
        hash_list=list(range(1,1000000))
        random.shuffle(hash_list)
        print("Created new hash list:",hash_list[0:10])    
    if map_table_or_dict is not None:
        if isinstance(map_table_or_dict,dict):
            if isinstance(map_table_or_dict.values()[0],list) and len(map_table_or_dict.values()[0])==2:
                print("Debug: Stripping previous hash table results")
                clean_dict={k: map_table_or_dict[k][0] for k in map_table_or_dict}
            elif len(map_table_or_dict.values()[0])==1:
                clean_dict=map_table_or_dict.copy()
            else:
                print("Mapping dictionary format not recognised, aborting")
                return
        elif isinstance(map_table_or_dict,pd.DataFrame):
            #TODO, probably worth doing more checks about the data coming, 
            print("Using ",map_table_or_dict.columns[0]," as key and ",map_table_or_dict.columns[1]," as anon key." )
            clean_dict=dict(zip(map_table_or_dict[map_table_or_dict.columns[0]], map_table_or_dict[map_table_or_dict.columns[1]]))
        else:
            print("Mapping object provided is not a Dictionary or a Dataframe, aborting")
            return
    else:
        #no mapping object provided
        clean_dict=None
    translation=_anonymise(df_list,key_list,clean_dict,hash_list)     
    if hash_list:
        #The translation is done to hash_list[anon_key] instead of just the
        #random anon_key. This is to further scramble the results esp when we 
        #add new items so we need to create that translation table.
        #also we can create a string based ID or some other criteria for a hash
        translation_list=[[k, translation[k],hash_table[translation[k]]] for k in  translation]
        translation_df=pd.DataFrame(data=translation_list,columns=["original_key","anon_key","hash_key"])
        print("Finished, returning translation DataFrame and hashtable as separate return values")
        return translation_df,hash_list
    else:
        translation_list=[[k,translation[k]] for k in translation]
        translation_df= pd.DataFrame(data=translation_list,columns=["original_key","anon_key"])
        print("Finished, returning translation DataFrame")
        return translation_df

translation=anonymise_dataframe([df1,df2],["mkey","mkey"])
print("Translation:",translation)
translation2,hash_list =anonymise_dataframe([df3],["mkey"],translation, create_new_hash_list=True)
print("Translation2:",translation2)


[nan, 'b', 'c', 'a', 'd', 'e']
Creating new mapping dictionary
New keys [nan, 'b', 'c', 'a', 'd', 'e']
Dummy keys [1, 2, 3, 4, 5, 6]
[2, 3, 5, 1, 6, 4]
Finished, returning translation DataFrame
Translation:   original_key  anon_key
0          NaN         2
1            b         3
2            c         5
3            a         1
4            d         6
5            e         4
Created new hash list: [423011, 13344, 85268, 514809, 756570, 851640, 8796, 124188, 51181, 798156]
Using  original_key  as key and  anon_key  as anon key.
['b', 'a', 'f', 'e']
New keys {'f'}
Dummy keys [7]
[7]
Finished, returning translation DataFrame and hashtable as separate return values
Translation2:   original_key  anon_key  hash_key
0          NaN         2    798924
1            b         3    606211
2            c         5    160441
3            a         1     31286
4            d         6    864256
5            e         4    771781
6            f         7    223889


In [17]:
hash_list[7]

124188

In [18]:
df3

,tkey,mkey,tvalue,mkey_anon
0,108,b,13.00,514809
1,109,a,19.04,13344
2,110,b,10.30,514809
3,111,b,50.00,514809
4,112,b,51.99,514809
5,113,e,60.00,756570
6,114,e,10.00,756570
7,115,f,4.20,124188
